### 드롭아웃 적용 여부 차이 비교 실습

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

#### 드롭아웃 적용한 신경망 모델 정의

In [2]:
class DropoutNet(nn.Module):
    def __init__(self) :
        super(DropoutNet, self).__init__()
        # 사용할 데이터 크기가 28x28 -> 784 1차원
        self.fc1 = nn.Linear(784, 500)
        self.dropout = nn.Dropout(p=0.5) # p=0.5, 50%
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x) :
        x = x.view(x.size(0), -1) # 입력 데이터 크기 조절 -> 1차원으로 펼침
        x = torch.relu(self.fc1(x)) # self.fc1(x) -> relu 적용
        x = self.dropout(x)
        x = self.fc2(x)
        return x

#### 드롭아웃 적용하지 않은 신경망 모델 정의

In [3]:
class NonDropoutNet(nn.Module) :
    def __init__(self) :
        super(NonDropoutNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x) :
        x = x.view(x.size(0), -1) 
        x = torch.relu(self.fc1(x)) 
        x = self.fc2(x)
        return x

#### MNIST 데이터셋 로드 및 전처리

In [4]:
train_transform = transforms.Compose([
    # transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.3,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.3,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=False, transform=train_transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=False, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#### 드롭아웃 적용 모델 초기화, loss, 옵티마이저 정의

In [5]:
# 드롭아웃 적용 모델 초기화
dropout_model = DropoutNet()
print(dropout_model)

# loss 함수, 옵티마이저 설정
dropout_criterion = nn.CrossEntropyLoss()
dropout_optimizer = optim.SGD(dropout_model.parameters(), lr=0.001)

DropoutNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


#### 드롭아웃 적용 모델 학습, 평가 정의

In [6]:
# 드롭아웃 적용 모델 학습 코드
for epoch in range(10) :
    # Train 모드 선언
    dropout_model.train() 
    for images, labels in train_loader :
        dropout_optimizer.zero_grad()
        dropout_output = dropout_model(images)
        dropout_loss = dropout_criterion(dropout_output, labels)
        dropout_loss.backward()
        dropout_optimizer.step()

# 드롭아웃 적용 모델 평가 코드
dropout_model.eval()
with torch.no_grad() :
    dropout_correct = 0 
    dropout_total = 0 
    for images, labels in test_loader :
        test_out = dropout_model(images)
        _, dropout_predict = torch.max(test_out.data, 1)
        dropout_total += images.size(0)
        dropout_correct += (dropout_predict == labels).sum().item()
    print(f"드롭아웃 적용 모델 정확도 : {(100 * dropout_correct / dropout_total):.2f}")

드롭아웃 적용 모델 정확도 : 91.22


#### 드롭아웃 적용되지 않은 모델 학습 평가 정의

In [7]:
# 모델 호출
non_dropout_model = NonDropoutNet()

# 옵티마이저 loss function 정의
non_dropout_criterion = nn.CrossEntropyLoss()
non_dropout_optimizer = optim.SGD(non_dropout_model.parameters(), lr=0.001)

# 드롭아웃 적용되지 않은 모델 학습
for epoch in range(10) :
    non_dropout_model.train()
    for images, labels in train_loader :
        non_dropout_optimizer.zero_grad()
        non_dropout_output = non_dropout_model(images)
        non_dropout_loss = non_dropout_criterion(non_dropout_output, labels)
        non_dropout_loss.backward()
        non_dropout_optimizer.step()

# 드롭아웃 적용하지 않은 모델 평가
non_dropout_model.eval()
with torch.no_grad() :
    non_dropout_correct = 0 
    non_dropout_total = 0
    for images, labels in test_loader :
        non_dropout_out = non_dropout_model(images)
        _, non_dropout_predict = torch.max(non_dropout_out.data, 1)
        non_dropout_total += images.size(0)
        non_dropout_correct += (non_dropout_predict == labels).sum().item()
    print(f"드롭아웃 적용 모델 정확도 : {(100 * non_dropout_correct / non_dropout_total):.2f}")

드롭아웃 적용 모델 정확도 : 91.23
